In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_lactate_flag,lab_lactate_first,lab_lactate_min,lab_lactate_max,lab_lactate_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1.4,1.4,1.4,0,0,1,0,0.5444486,1.836721
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2569337,1.345775
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1.6,1.6,2.6,1,0,1,0,0.3264142,3.063592
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,3.8,2.2,3.8,1,1,1,0,0.8063102,1.240217
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.2239520,1.288580
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,1,1.4,1.4,1.4,0,0,1,0,0.3457691,2.892104


In [4]:
wtd_pval <- data.table::fread(file.path(data_dir, "wtd_pval.csv"), data.table = FALSE)
head(wtd_pval)

cov,pval
age,0.062
gender,0.471
first_careunit,0.154
weight,0.164
elix_score,0.098
saps,0.127


In [5]:
wpv <- wtd_pval %>%
    filter(pval < 0.05) %>%
    pull(cov)

wpv

[1] "sofa"                       "vent"                      
[3] "vaso"                       "sedative"                  
[5] "icd_chf"                    "icd_afib"                  
[7] "lab_chloride_first"         "lab_troponin_flag"         
[9] "lab_creatinine_kinase_flag"

In [6]:
ipw_svydesign <- readRDS(file.path(data_dir, "ipw_svydesign.rds"))

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + sofa + vent + vaso + sedative + icd_chf + icd_afib + lab_chloride_first + lab_troponin_flag + lab_creatinine_kinase_flag"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = ipw_svydesign)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = ipw_svydesign)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.427152   0.488112  -2.924 0.003470 ** 
echo1                       -0.237345   0.069800  -3.400 0.000677 ***
sofa                         0.205440   0.010885  18.874  < 2e-16 ***
vent1                        0.379503   0.106069   3.578 0.000349 ***
vaso1                       -0.003081   0.081146  -0.038 0.969715    
sedative1                   -0.464683   0.105961  -4.385 1.18e-05 ***
icd_chf1                     0.062565   0.078827   0.794 0.427399    
icd_afib1                    0.365411   0.077822   4.695 2.72e-06 ***
lab_chloride_first          -0.008372   0.004549  -1.840 0.065769 .  
lab_troponin_flag1           0.416809   0.098269   4.242 2.25e-05 ***
lab_creatinine_kinase_flag1 -0.217311   0

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.2399915,0.09219644,0.6247088
echo1,0.7887192,0.68787280,0.9043504
sofa,1.2280652,1.20214355,1.2545457
vent1,1.4615574,1.18721727,1.7992916
vaso1,0.9969238,0.85033602,1.1687816
sedative1,0.6283342,0.51050041,0.7733663
icd_chf1,1.0645639,0.91216623,1.2424230
icd_afib1,1.4411055,1.23723993,1.6785629
lab_chloride_first,0.9916626,0.98285969,1.0005445
lab_troponin_flag1,1.5171121,1.25132764,1.8393497
